# Imports

In [1]:
import sys
from pathlib import Path

sys.path.extend(['/Users/fabianloewe/Projects/master-thesis/tools/aletheia'])

import pandas as pd
import aletheialib.attacks as attacks

# Prepare data

In [2]:
EMBEDDING_METHODS = [
    'MobiStego', 'Passlok', 'Pictograph', 'PixelKnot', 'PocketStego', 'SteganographyM'
]

BASE_PATH = Path('../master-thesis/datasets/StegoAppDB_stegos_20240309-030352')

df = pd.read_csv(BASE_PATH / 'StegoAppDB_stegos_20240309-030352_stego_directory.csv')

dfs = {m: df[df['embedding_method'] == m] for m in EMBEDDING_METHODS}
dfs_stegos = {
    m: list(BASE_PATH / 'stegos' / si for si in dfs[m]['image_filename'])
    for m in EMBEDDING_METHODS
}

# Extract LSBs

In [3]:
STEGOAPPDB_PATH = Path('../datasets/StegoAppDB_stegos_20240309-030352')
INFO_FILE = STEGOAPPDB_PATH / 'StegoAppDB_stegos_20240309-030352_stego_directory.csv'
METHOD_COLUMN = 'embedding_method'
MESSAGE_DIR = Path('../datasets/StegoAppDB_stegos_20240309-030352/message_dictionary')

CONFIGS = [
    ('MobiStego', 'RGB', 2, 'big', 'row'),
    ('PocketStego', 'B', 1, 'big', 'col'),
]

info_file = pd.read_csv(INFO_FILE)
embedding_methods = info_file[METHOD_COLUMN].unique()

def find_nth_substring(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start + len(needle))
        n -= 1
    return start

def get_original_message(stego_img):
    img_row = info_file[info_file['image_filename'] == stego_img.name]
    msg_name = img_row['message_dictionary'].values[0]
    starting_line_index = img_row['message_starting_index'].values[0]
    msg_len = img_row['message_length'].values[0]
    full_msg = (MESSAGE_DIR / msg_name).read_text()
    start_index = find_nth_substring(full_msg, '\n', starting_line_index - 1) + 1
    return full_msg[start_index:start_index + msg_len].encode('utf-8')

all_lsbs = {
    method: [(get_original_message(stego), attacks.lsb_extract(
        stego,
        channels=channels,
        bits=bits,
        endian=endian,
        direction=direction
    )) for stego in dfs_stegos[method]]
    for method, channels, bits, endian, direction in CONFIGS
}

In [5]:
print('Start:')
for original, extracted in all_lsbs['MobiStego']:
    print(extracted[:10].tobytes())
    
print('\nEnd:')
for original, extracted in all_lsbs['MobiStego']:
    print(extracted[3+len(original):10+3+len(original)].tobytes())

b'@!#Which i'
b'@!#He shal'
b'@!#Most ra'
b'@!#Nor ten'
b'@!#Nor wil'
b'@!#The nat'
b'@!#O, here'
b'@!#HASTING'
b'@!#I think'
b'@!#Come Ca'
b'@!#POMPEY\n'
b'@!#Sever t'
b'@!#Is she '
b'@!#In the '
b'@!#ANTONIO'
b'@!#Nay, to'
b'@!#ANTONIO'
b'@!#QUEEN\nW'
b'@!#lady? h'
b'@!#not me:'
b'@!#SICINIU'
b'@!#your gr'
b'@!#That la'
b'@!#Sir, I '
b'@!#I must '
b'@!#Acts li'
b'@!#that I '
b'@!#Into a '
b'@!#I will '
b'@!#Soothsa'
b'@!#KENT\nHe'
b'@!#As who '
b'@!#Truly, '
b'@!#If stud'
b'@!#all oth'
b'@!#KING RI'
b'@!#Sir,\nFo'
b'@!#FERDINA'
b'@!#Which e'
b'@!#In view'
b'@!#This is'
b'@!#That sh'
b'@!#POSTHUM'
b'@!#If what'
b'@!#Forgets'
b'@!#And to '
b'@!#Told th'
b'@!#And tre'
b'@!#And we '
b'@!#POMPEY\n'
b'@!#With su'
b'@!#If you '
b'@!#Subdued'
b'@!#Yet nei'
b'@!#PROSPER'
b'@!#Be born'
b"@!#'Tis ve"
b'@!#I tell '
b'@!#FERDINA'
b'@!#Exeunt '
b'@!#Very gl'
b'@!#Enter A'
b'@!#No care'
b'@!#VIRGILI'
b'@!#CLEON\nB'
b'@!#This Ed'
b'@!#thy old'
b'@!#He call'
b'@!#Enter D'
b'@!#KING HE'
b'@!#I have '


In [8]:
print('Start:')
for original, extracted in all_lsbs['PocketStego']:
    print(extracted[:10].tobytes())
  
print('\nEnd:')  
for original, extracted in all_lsbs['PocketStego']:
    print(extracted[len(original):10+len(original)].tobytes())

Start:
b'CYMBELINE\n'
b'Without de'
b'A weeder-o'
b'Speak on w'
b"Error i' t"
b'He shall n'
b'DAVY\nI bes'
b'Which died'
b'And bear i'
b'But being '
b'HENRY BOLI'
b'Will be hi'
b'I humbly d'
b'Murdering '
b'Might yet '
b'one be a m'
b'Thou gaves'
b"'Twas a ro"
b'PORTIA\nHer'
b'He hath st'
b'O, in a to'
b'ADRIANO DE'
b'there a wi'
b'And dare n'
b'Or as the '
b'Exit\nEnter'
b'Enter ANTI'
b'Me?\nFERDIN'
b'Or if that'
b'The cocker'
b'CONSTANCE\n'
b'LEONATO\nNi'
b'What frien'
b'Entirely t'
b'Yes Shyloc'
b'Not to be '
b'Thanks, sw'
b'And light '
b'He makes m'
b"Is as 'twe"
b'My famous '
b'New horror'
b'for turnin'
b'Lest men s'
b'PRINCE HEN'
b'make not t'
b'LYSIMACHUS'
b'LAERTES\nTh'
b'AARON\nSome'
b'For I have'
b'MOROCCO\nTh'
b'PRINCESS\nT'
b'wrongs you'
b'To what tu'
b'OSWALD\nWhy'
b'Chamberlai'
b'To castiga'
b'Even now, '
b'ANTIPHOLUS'
b'As frankly'
b'The mighty'
b'[Advancing'
b'And be tho'
b'Exit Provo'
b'Which I wi'
b'An unknown'
b'Currents t'
b'And think '
b'Good my lo'
b'I will wit'
b